In [5]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
# from pandas import DataFrame
import requests
import json
import re
import sys
import pandas as pd

In [14]:
#Prep for looping over all pages.
i=0
url = "https://www.ictergezocht.nl/it-vacatures/?pnr=" + str(i)

#Creating storage for URLS
links1 = [] #Create open list to store links later. non spot
links2 = [] #Create open list to store links later. spotlight vacancies
links3 = [] #Both attempt <- Works!



while True:
    i = i+1
    if i == 75: #make sure it stops at last page. 
        break

    #should stop the code in case of error
    page = requests.get(url)
    if page.status_code != 200:
        break

    url = "https://www.ictergezocht.nl/it-vacatures/?pnr=" + str(i)
    print(url)
    #Data retrieval prepetaion
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()

    page_soup1 = soup(page_html,"html.parser")

    #Grabs each block (spot and non spot)
    content_blocks = page_soup1.findAll("div",{"class":"content_block vacitem"})

    spot_content_blocks = page_soup1.findAll("div",{"class":"content_block vacitem spotlight "})

    #Grab specific info for each block
    aaa = 0
    for content_block in content_blocks:
        aaa = aaa + 1
        
        t = content_block.find("a",{"class":"title"})
        try:
            link = t.get('href')
        except Exception:
            pass

        #title_block = content_block.findAll("a",{"class":"title"})
        #func_title = title_block[0].text

        links1.append(link) #My attempt to save everything in the same list
        links3.append(link) #Attempt all
        print(aaa)
        #print("Job title: "+ func_title)
        #print(link)


    
    for spot_content_block in spot_content_blocks:
        aaa = aaa +1
        t = spot_content_block.find("a",{"class":"title"})
        try:
            link = t.get('href')
        except Exception:
            pass
        
        
        #spot_title_block = spot_content_block.findAll("a",{"class":"title"})
        #spot_func_title = spot_title_block[0].text

        links2.append(link) #My attempt to save everything in the same list
        links3.append(link) #Attempt all
        print(aaa)
        #print("Spotlight Job title: "+ spot_func_title)
        #print(link)


#############################################################################
print("DONE stage1! Retrieving Links to Job descriptions")
#############################################################################

https://www.ictergezocht.nl/it-vacatures/?pnr=1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
https://www.ictergezocht.nl/it-vacatures/?pnr=2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
https://www.ictergezocht.nl/it-vacatures/?pnr=3
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
https://www.ictergezocht.nl/it-vacatures/?pnr=4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
https://www.ictergezocht.nl/it-vacatures/?pnr=5
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
https://www.ictergezocht.nl/it-vacatures/?pnr=6
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
https://www.ictergezocht.nl/it-vacatures/?pnr=7
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
https://www.ictergezocht.nl/it-vacatures/?pnr=8
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
https://www.ictergezocht.nl/it-vacatures/?pnr=9
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
https://www.ictergezocht.nl/it-vacatures/?pnr=10
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
DONE stag

In [35]:
print(len(links1), len(links2), len(links3))

1337 123 1460


In [15]:
#Preparing CSV
ict_old = pd.read_csv("ict-vacatures-text-all.csv")
filename = "placeholder"
topics_dict = { "Job_title":[], \
                "Company":[], \
                "Location":[], \
                "Offer":[], \
               "Link":[], \
                "Posted":[], \
                "Text":[], \
                }


urls = links3
test_list = []
for url in urls:
    print(url)
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()

    vac_soup1 = soup(page_html,"html.parser")
    data = vac_soup1.find("script", type="application/ld+json").text
    data = ' '.join(re.sub(r"(\n|\t)*", "", data).split()) # --->>> REMOVE NEW LINE CHARS AND WHITESPACES

    # --->>> PREPROCESSING DATA TO REPLACE HTML CODES WITH SIGNS
    data = re.sub(r"\&amp;", "&", data) # --->>> AMPERSAND '&'
    data = re.sub(r"\&reg;", "", data) # --->>> REGISTERED SIGN '®'

    # --->>> IMPORTANT <<<--- #
    # --->>> APPARENTLY SOUP MAKES SOMETIMES A MISTAKE WITH WRITING ANOTHER COMMA AT THE END OF A BLOCK
    # --->>> WHERE NO MORE BLOCKS ARE COMING AND JSON DOES NOT LIKE THAT...
    data = re.sub(r"\},\s\}", "} }", data) # --->>> REMOVE TRAILING COMMA IF NO MORE BLOCKS ARE COMING

    # --->>> END PREPROCESSING

    # --->>> I USED THIS TO FIND THE ERRORS AT RUNTIME
    try:
        newDictionary = json.loads(data) # --->>> LOAD EVERYTHING TO A DICTIONARY
    except (Exception, ArithmeticError) as e:
        template = "An exception of type {0} occurred. Arguments:\n{1!r}" # --->>> ERROR MESSAGE TEMPLATE
        message = template.format(type(e).__name__, e.args) # --->>> COMPILE ERROR MESSAGE
        print (message) # --->>> PRINT THE ERROR
        print(data) # --->>> PRINT DATA TO CHECK WHAT COULD CAUSE ERROR
        sys.exit("JSONDecodeError") # --->>> STOP EXECUTION OF SCRIPT

    # --->>> THE VALUES WE ARE INTERESTED IN
    # --->>> JUST ADD MORE IF YOU NEED
    # --->>> USE print(newDictionary) TO FIND THE INDICES YOU ARE INTERESTED IN
    # print(newDictionary)
    title = newDictionary['title']
    company = newDictionary['hiringOrganization']['name']
    offer = newDictionary['@type']
    location = newDictionary['jobLocation']['address']['addressLocality']
    date = newDictionary['datePosted']

    #Preparing for CSV
    title.replace(",",";")
    company.replace(",",";")
    offer.replace(",",";")
    location.replace(",",";")
    date.replace(",",";")    
    #Writing to Pandas frame
    topics_dict["Job_title"].append(title)
    topics_dict["Company"].append(company)
    topics_dict["Location"].append(location)
    topics_dict["Offer"].append(offer)
    topics_dict["Link"].append(url)
    topics_dict["Posted"].append(date)

    all_text = []
    vactexts = vac_soup1.findAll("div",{"class":"vactext"}) 
    for vactext in vactexts:
        tekst = vactext.text
        tekst = re.sub(r"[\n\t]*", "", tekst)

        all_text.append(tekst)

        str_text = ''.join(all_text) 
        str_text = re.sub(r'[\n\t]',' ', str_text)
    # --->>> END PREPROCESSING

    # --->>> WRITE VALUES TO FILE
    str_text.replace(",",";")
    topics_dict["Text"].append(str_text)



#############################################################################
print("DONE stage2! Retrieving the data")
#############################################################################

https://www.ictergezocht.nl/5627_sqout-bv/ict-vacatures/36733_software-tester/
https://www.ictergezocht.nl/5627_sqout-bv/ict-vacatures/36732_java-tester/
https://www.ictergezocht.nl/5627_sqout-bv/ict-vacatures/36731_software-tester/
https://www.ictergezocht.nl/5627_sqout-bv/ict-vacatures/36730_testautomatiseerder/
https://www.ictergezocht.nl/5627_sqout-bv/ict-vacatures/36729_test-analist/
https://www.ictergezocht.nl/3632_adver-online-bv/ict-vacatures/36728_applicatiebeheerder/
https://www.ictergezocht.nl/2634_careervalue-bv/ict-vacatures/36727_medior-senior-software-tester-in-webomgeving-gezocht/
https://www.ictergezocht.nl/2634_careervalue-bv/ict-vacatures/36726_embedded-cplusplus-is-jouw-tweede-naam/
https://www.ictergezocht.nl/2634_careervalue-bv/ict-vacatures/36725_top-werkgever-zoekt-test-manager/
https://www.ictergezocht.nl/2634_careervalue-bv/ict-vacatures/36724_javascript-kanjer-gezocht/
https://www.ictergezocht.nl/2634_careervalue-bv/ict-vacatures/36723_marketing-specialist-in

https://www.ictergezocht.nl/6098_en-hr-solutions-bv/ict-vacatures/36653_ict-architect/
https://www.ictergezocht.nl/127_aatop-ict/ict-vacatures/36652_c-sharp-engineer/
https://www.ictergezocht.nl/127_aatop-ict/ict-vacatures/36651_senior-systeembeheerder-intern-ede/
https://www.ictergezocht.nl/8231_veiligheidsregio-noord-holland-noord/ict-vacatures/36649_technisch-applicatiebeheerder/
https://www.ictergezocht.nl/3040_ict-career-bv/ict-vacatures/36647_junior-applicatie-en-systeem-engineer-bij-basefarm/
https://www.ictergezocht.nl/4375_humint/ict-vacatures/36646_solution-architect/
https://www.ictergezocht.nl/5359_korndewal/ict-vacatures/36645_product-owner-re-vision-waardenburg/
https://www.ictergezocht.nl/2634_careervalue-bv/ict-vacatures/36643_unieke-start-up-in-amsterdam-zoekt-java-talenten/
https://www.ictergezocht.nl/2634_careervalue-bv/ict-vacatures/36642_netwerk-engineer-gezocht-voor-topclub/
https://www.ictergezocht.nl/2634_careervalue-bv/ict-vacatures/36641_gezocht-security-offic

https://www.ictergezocht.nl/127_aatop-ict/ict-vacatures/36565_1e-lijns-support-engineer-junior-starter/
https://www.ictergezocht.nl/5340_xiphos-bv/ict-vacatures/36564_skilled-servicedesk-medewerker-utrecht-doorgroeimogelijkheid-naar-systeembeheer/
https://www.ictergezocht.nl/1186_vonq/ict-vacatures/36584_functioneel-beheerder/
https://www.ictergezocht.nl/7992_connectingthedots/ict-vacatures/36568_fullstack-web-developer-zwolle-fulltime-kans-op-vaste-baan/
https://www.ictergezocht.nl/2634_careervalue-bv/ict-vacatures/36563_java-j2me-en-java-me-mobiele-app-ontwikkelaar/
https://www.ictergezocht.nl/4881_sourceselect/ict-vacatures/36561_applicatiebeheerder-warehouse-management-system-wms-in-rotterdam/
https://www.ictergezocht.nl/2634_careervalue-bv/ict-vacatures/36560_junior-php-developer-e-commerce/
https://www.ictergezocht.nl/2634_careervalue-bv/ict-vacatures/36559_gezocht-technisch-microsoft-engineer-in-de-betuwe/
https://www.ictergezocht.nl/127_aatop-ict/ict-vacatures/36558_junior-supp

In [19]:
data

'{ "@context": "http://schema.org", "@type": "JobPosting", "title": "Head of Department ICT", "datePosted": "2018-07-09", "validThrough": "2018-11-09", "description": "The office of MSF-Holland is located in Amsterdam and hosts ca. 350 employees, who support MSF-OCA field projects.<br \\/>\\nMSF-Holland\\/MSF-OCA is looking for a motivated, experienced candidate for the position of:<br \\/>\\n<br \\/>\\n<strong>Head of Department ICT<\\/strong><br \\/>\\n<br \\/>\\n(40 hours per week) \\/ based in Amsterdam<br \\/>\\nThe ICT department in MSF-Holland\\/MSF-OCA consists of approximately 27 people. The head of Department reports to the Director of Resources and oversees ICT in Amsterdam office as well as our OCA missions around the globe and line manages directly the Service Delivery Manager and the Project and Application coordinator.<br \\/>\\n<br \\/>\\n<strong>Responsibilities<\\/strong>\\n<ul>\\n\\t<li>Formulate, drive and implements the overall ICT strategy and vision for MSF OCA;<

In [18]:
ict_new = pd.DataFrame(topics_dict)
ict_all = ict_old.append(ict_new, sort=False, ignore_index=True)

ict_all.columns = ict_all.columns.str.strip()
ict_all.columns.tolist()

ict_all = ict_all.drop_duplicates(['Link'])
ict_all.to_csv("ict-vacatures-text-all.csv", encoding = "utf-8", index= False)

print(len(ict_new), len(ict_old), len(ict_all), sep='\n')

200
1475
1502
